In [2]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import os
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()
embeddings = AzureOpenAIEmbeddings(
openai_api_type="azure",
openai_api_version=os.environ["OPENAI_API_EMBEDDING_VERSION"],
openai_api_key=os.environ["OPENAI_API_EMBEDDING_KEY"],
azure_endpoint=os.environ["AZURE_OPENAI_EMBEDDING_ENDPOINT"],
deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"],
model=os.environ["AZURE_OPENAI_EMBEDDING_MODEL"],
validate_base_url=True,
)

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
from langchain.tools.retriever import create_retriever_tool
vector_store = PineconeVectorStore(index=pc.Index("langstuffindex"), embedding=embeddings)

d:\agentagent\AgentAgent\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain.tools.retriever import create_retriever_tool
retriever = vector_store.as_retriever()

In [5]:
from bs4 import BeautifulSoup
import html2text
import httpx

def fetch_documents(url: str) -> str:
    """Fetch a document from a URL and return the markdownified text.

    Args:
        url (str): The URL of the document to fetch.

    Returns:
        str: The markdownified text of the document.
    """
    httpx_client = httpx.Client(follow_redirects=True, timeout=10)

    try:
        response = httpx_client.get(url, timeout=10)
        response.raise_for_status()
        html_content = response
        soup = BeautifulSoup(html_content, 'html.parser')
    
        img_tags = soup.find_all('img')
        for img_tag in img_tags:
            img_tag.decompose()

        target_div = soup.find('div', class_= "theme-doc-markdown markdown") #langchain
        
        if not target_div:
            target_div = soup.find('article') #langraph

        if not target_div:
            target_div = soup.find('html') #langraph

        if not target_div:
            return html2text.html2text(str(soup))
        
        return html2text.html2text(str(target_div))
    except (httpx.HTTPStatusError, httpx.RequestError) as e:
        return f"Encountered an HTTP error: {str(e)}"

In [6]:
from langchain import hub
from langchain_openai import ChatOpenAI

from langgraph.prebuilt import create_react_agent

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-4o-mini")


# AGENT state creation

In [7]:
from pydantic import BaseModel, Field
from langgraph.graph import  MessagesState
class NodeBuilderState(MessagesState):
    """State for the node builder."""
    next: str = Field(description="Next node to go to")
    schema_info: str = Field(description="Schema information about the node")
    input_schema: str = Field(description="Input schema of the node")
    output_schema: str = Field(description="Output schema of the node")
    description: str = Field(description="Description of the node")
    function_name: str = Field(description="Function name of the node")
    code: str = Field(description="Code for the node")
    toolset: list[str] = Field(description="List of tools to be used in the node by the llm")
    node_type : str = Field(description="Type of node, deterministic if the function is deterministic and requires simple python code generation, ai if the function is not deterministic and requires a llm usage for meeting the requirements")
    pass


# NODE type identification and deterministic code gen

In [108]:
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal
from langchain_core.messages import HumanMessage


class NodeType(BaseModel):
    node_type: Literal["deterministic", "ai"] = Field(description=
                                                      """Type of node, 
                                                      : deterministic if the function is deterministic and requires simple python code generation,
                                                      : ai if the function is not deterministic, like analysis of input, plan generation or any other thing which is fuzzy logic requires Artificial intelligencefor meeting the requirements""")


def determine_node_type(state: NodeBuilderState):
    """Determine the type of node."""
    node_type: str = state["node_type"]
    if node_type == "deterministic":
        return "deterministic_code"
    elif node_type == "ai":
        return "ai_node_gen_supervisor"

node_info_prompt= """
You are provided with the following information about the node:
<SchemaInfo>
{schema_info}
</SchemaInfo>
<InputSchema>
{input_schema}
</InputSchema>
<OutputSchema>
{output_schema}
</OutputSchema>
<Description>
{description}
</Description>
<FunctionName>
{function_name}
</FunctionName>

Below is the skeleton of the function that you need to implement:
def {function_name}(state:{input_schema}) -> {output_schema}:
    \"\"\"{description}\"\"\"
    # Implement the function to meet the description.
    
the state is of type {input_schema} and the function is of type {output_schema}
The general idea is that the implementation would involve extracting the input from the state, and updating the state with the output. Description contains the logic for this blackbox
"""
    
def identify_node(state: NodeBuilderState):
    """Identify the node and return the information."""
    # Extract the information from the state
    llm_with_structured_output = llm.with_structured_output(NodeType)
    prompt = ChatPromptTemplate.from_template(
        """You are supposed to identify the type of node based on the information provided.
            <SchemaInfo>
            {schema_info}
            </SchemaInfo>
            <InputSchema>
            {input_schema}
            </InputSchema>
            <OutputSchema>
            {output_schema}
            </OutputSchema>
            <Description>
            {description}
            </Description>
            <FunctionName>
            {function_name}
            </FunctionName>
""")
    type_of_node = llm_with_structured_output.invoke(prompt.format(
        schema_info=state["schema_info"],
        input_schema=state["input_schema"],
        output_schema=state["output_schema"],
        description=state["description"],
        function_name=state["function_name"]))
    return {"node_type": type_of_node.node_type, "messages": [HumanMessage(content=node_info_prompt.format(
        schema_info=state["schema_info"],
        input_schema=state["input_schema"],
        output_schema=state["output_schema"],
        description=state["description"],
        function_name=state["function_name"])) ]}

def deterministic_code_gen(state: NodeBuilderState):
    """Generate the code for the node."""
    prompt = ChatPromptTemplate.from_template(
        """Generate the python code for the function {function_name}.
You are provided with the following information about the node:
The schema information is as follows: {schema_info}
The input schema is: {input_schema}
The output schema is: {output_schema}
The description of the function is: {description}

Implement the function to meet the requirements.
""")
    code = llm.invoke(prompt.format(
        schema_info=state["schema_info"],
        input_schema=state["input_schema"],
        output_schema=state["output_schema"],
        description=state["description"],
        function_name=state["function_name"]))
    return {"code": code.content}


# Supervisor creation function

In [109]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from typing import  Literal, TypedDict
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import SystemMessage, AIMessage


def make_supervisor_node(llm: BaseChatModel, members: list[str]) -> str:
    options = ["FINISH"] + members
    system_prompt = (
        f"""You are a supervisor tasked with managing a conversation between the
        following workers: {members}. Given the following user request,
        respond with the worker to act next. Each worker will perform a
        task and respond with their results and status. When finished,
        respond with FINISH.
        
        As a supervisor you need to identify which workers to call and the sequence of calling them. 
        Your goal is to build a langgraph node. 
        
        <NodeBuildingStrategies>
        1. prompting + structured_output: 
            a. Decision making nodes: the node's objective is to analyze the input and use structured output for decision making. Example: analyze the sentiment of the user_query, return llm output as a structured class.
        2. Use prompting + Interrupt: analyze the input using LLM with an appropriate prompt, use interupt to get human approval or input. 
        3. Use prompting + tool-calling:
            a. if the node's function is to answer a user_query which needs factual information to answer
            b. if the user_query is to make calls to downstream APIs the inputs of which are determined by llm(model) 
            c. Whenever the node involves a LLM(model) needing sensors or actuators, for information retrieval, and allowing llm(model) to 'DO' stuff
            d. One of the best use cases of this technique is when you know that there would be 'n' different tools which could be used in different scenarios to handle a query in a particular domain.
        </NodeBuildingStrategies>
        
        Select the best nodebuilding strategy to meet the functional requirements of the node. 

        Once you have queried enough information, respond with 'compile_code' at last to orchestrate all information from different workers into a final prodict. 
        That is when your task will be finished then respond with FINISH
        """
    )

    class Router(BaseModel):
        """Worker to route to next. If no workers needed, route to FINISH."""
        reason: str = Field(description="Analysis of supervisor, justification of selecting the next worker, why exactly is this particular worker selected and expectations from it.")
        next: Literal[*options] = Field(description="worker to act next") # type: ignore

    def supervisor_node(state: NodeBuilderState) -> Command[Literal[*members, "__end__"]]:
        """An LLM-based router."""
        messages = [
            {"role": "system", "content": system_prompt},
        ] + state["messages"]
        response = llm.with_structured_output(Router).invoke(messages)
        goto = response.next
        if goto == "FINISH":
            goto = END

        return Command(goto=goto, update={"next": goto, "messages": [AIMessage(content= response.reason)]})

    return supervisor_node


# TOOLSETGEN agent

In [110]:
from langchain_core.messages import HumanMessage


toolset_retrieval_prompt = """
    User will provide information about a node, Your task is to see if meeting the requirements of the node needs llm(aka model)'s tool calling capability.
    
    Information about tool calling capabilities of LLMs, how to create tools and how to bind them with a LLM is provided below:
    <TOOL_CALLING_EXAMPLE>
Example 1: 
``` python
from typing import Literal
from langgraph.graph import StateGraph, MessagesState, START, END
from langchain_core.messages import AIMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

@tool
def get_weather(location: str):
    \"\"\"Call to get the current weather.\"\"\"
    if location.lower() in ["sf", "san francisco"]:
        return "It's 60 degrees and foggy."
    else:
        return "It's 90 degrees and sunny."


@tool
def get_coolest_cities():
    \"\"\"Get a list of coolest cities\"\"\"
    return "nyc, sf"

tools = [get_weather, get_coolest_cities]
tool_node = ToolNode(tools)

def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END


def call_model(state: MessagesState):
    \"\"\"This node answers questions related to weather using a varied weather related toolset\"\"\"
    messages = state["messages"]
    response = model_with_tools.invoke([SystemMessage(content= "Please analyze the following weather-related query and provide a detailed response with relevant information: You have tools like get_weather and get_coolest_cities to help you answer the queries" )] + messages)
    return {"messages": [response]}


workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, ["tools", END])
workflow.add_edge("tools", "agent")

app = workflow.compile()
```
***Dry run***

================================[1m Human Message [0m=================================

what's the weather in sf?
==================================[1m Ai Message [0m==================================

[{'text': "Okay, let's check the weather in San Francisco:", 'type': 'text'}, {'id': 'toolu_01LdmBXYeccWKdPrhZSwFCDX', 'input': {'location': 'San Francisco'}, 'name': 'get_weather', 'type': 'tool_use'}]
Tool Calls:
  get_weather (toolu_01LdmBXYeccWKdPrhZSwFCDX)
 Call ID: toolu_01LdmBXYeccWKdPrhZSwFCDX
  Args:
    location: San Francisco
=================================[1m Tool Message [0m=================================
Name: get_weather

It's 60 degrees and foggy.
==================================[1m Ai Message [0m==================================

The weather in San Francisco is currently 60 degrees with foggy conditions

</TOOL_CALLING_EXAMPLE>

<Output> 
Identify if tool_calling is relevant:
if yes: identify what kind of tools may be needed to be used by a LLM, and then write code for the llm binding functionality
if no: just respond no tool_calling needed.
</Output>
    """

def toolset_generation(state: NodeBuilderState) -> Command[Literal["ai_node_gen_supervisor"]]:
    """Generate the code for the node."""
    result = llm.invoke([SystemMessage(content=toolset_retrieval_prompt)] + state["messages"])
    return Command(
        update={
            "messages": [
                HumanMessage(content=result.content, name="toolset_suggester")
            ]
        },
        goto="ai_node_gen_supervisor",
    )


# PROMPTGEN agent

In [111]:
from langchain_core.messages import HumanMessage

prompt_vector_store = PineconeVectorStore(index=pc.Index("promptguide"), embedding=embeddings)
prompt_retriever = prompt_vector_store.as_retriever()
prompt_gen_retriever = create_retriever_tool(prompt_retriever, "Retrieve_info_on_prompting", "Search information about what are different prompting techniques relevant to the user requirements.")

promptgen_prompt = ChatPromptTemplate.from_messages([
         ("system", """
    You are a ReAct (Reasoning and Act) agent.
    You are tasked with generating a prompt to meet the objectives of the langgraph node.
    The langgraph node information is provided. 
    
    For example: 
    User query: the node is supposed to generate a plan using llms
    Thought: I need to generate a prompt that will make the LLM generate a plan for the given task.
    Action: Use the  'Retrieve_info_on_prompting' tool to search for plan-and-execute prompting techniques.
    Observation: I found a plan-and-execute prompting technique that can generate a plan for the given task.
    Action: I will customize the observed prompt to meet the requirements of the node.
    
    IMPORTANT: Your final output will be only a prompt, no code
    """),
    ("placeholder", "{messages}"),
    ])
prompt_gen_agent = create_react_agent(llm, tools=[prompt_gen_retriever], prompt = promptgen_prompt)

def prompt_generation(state: NodeBuilderState) -> Command[Literal["ai_node_gen_supervisor"]]:
    """Generate the code for the node."""
    response = prompt_gen_agent.invoke({"messages": state["messages"]})
    return Command(
        update={
            "messages": [
                HumanMessage(content=response["messages"][-1].content, name="prompt_generator")
            ]
        },
        goto="ai_node_gen_supervisor",
    )


# STRUCTUTRED OUTPUT GEN

In [112]:


struc_output_prompt = """
    User will provide you with the information about the node,  you are supposed to analyze the information and see if it requires LLM (aka model)'s 'with_structured_output()' function.

Here is information about structured outputs, why and when to use structured outputs:
<AboutStructuredOutput>
Structured output is beneficial in situations where consistent and verifiable data formats are needed, especially when integrating with databases, APIs, or building complex workflows. It helps reduce hallucinations, simplifies prompting, and enables reliable type-safety, making applications more predictable and easier to evaluate. 

Here's a more detailed breakdown:
1. Reduced Hallucinations: By enforcing adherence to a JSON Schema, structured outputs minimize the chance of models generating incorrect or irrelevant data. 
2. Simplified Prompting: You don't need overly complex or specific prompts to get consistently formatted output, as the schema provides the structure. 
3. Reliable Type-Safety: Structured outputs ensure that the model always generates data that fits the defined schema, eliminating the need for validation or retries due to format errors. 
4. Building Complex Workflows: Structured outputs are particularly useful for building multi-step workflows where the output of one step serves as input for the next. 
5. Integration with Databases and APIs: When integrating with systems that require structured data formats (like databases or APIs), structured outputs ensure consistency and avoid integration problems. 
6. Function Calling and Data Extraction: Structured outputs are recommended for function calling and extracting structured data from various sources. 
7. Consistent and Verifiable Output: The predictable format of structured outputs makes it easier to test, debug, and evaluate applications that rely on them. 
8. Explicit Refusals: You can now detect model refusals programmatically when using structured outputs, as the model will explicitly return a structured error message instead of a text-based one. 

In essence, structured output provides a robust and predictable way to manage the output of language models, making them more reliable and easier to integrate into various applications. 
</AboutStructuredOutput>

Here is an example showing how to use 'with_structured_output' method is below:
<StructuredOutputExample>
Here is an example of how to use structured output. In this example, we want the LLM to generate and fill up the pydantic class Joke, based on user query. 
``` python
from typing import Optional
from pydantic import BaseModel, Field


# Pydantic class for structured output
class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )

class JokeBuilderState(MessagesState):
    joke: Joke = Field(description= "joke generated by the GenerateJoke node.")

def GenerateJoke(state: JokeBuilderState):
    structured_llm = llm.with_structured_output(Joke)
    joke: Joke = structured_llm.invoke("Tell me a joke about cats")
    return { "joke": joke }
```

Output:
Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!', rating=7)
</StructuredOutputExample>
    
<Notes> 
1. The structured output pydantic class is not to be the same as the input_schema or output_schema, need to be specific
2. Do not hallucinate or write your own code to implement structured output, refer to 'StructuredOutputExample' section
</Notes>
    
<Output>
Check if the node needs llm's structured output functionality based on 'AboutStructuredOutput' section.
If yes, you are supposed to generate the code for the structured output functionality for the llm to use. 
If not needed, just mention there is no need for structured output functionality.
</Output>
    """

def structured_output_generation(state: NodeBuilderState) -> Command[Literal["ai_node_gen_supervisor"]]:
    """Generate the code for the node."""
    # fetch_documents("https://python.langchain.com/docs/how_to/structured_output/")
    response = llm.invoke( [SystemMessage(content=struc_output_prompt)]  + state["messages"])

    return Command(
        update={
            "messages": [
                HumanMessage(content=response.content, name="struct_output_generator")
            ]
        },
        goto="ai_node_gen_supervisor",
    )


# Interrupt gen

In [113]:

interrupt_gen_prompt: str = """
    User will provide you with the information about the node, you are supposed to analyze the information and see if it requires interrupt functionality.
    
    Following is the information about the interrupt functionality, what is the purpose of interrupt, design patterns, how to implement them:
    <InterruptInfo>
    The interrupt function in LangGraph enables human-in-the-loop workflows by pausing the graph at a specific node, presenting information to a human, and resuming the graph with their input. This function is useful for tasks like approvals, edits, or collecting additional input. The interrupt function is used in conjunction with the Command object to resume the graph with a value provided by the human.


``` python
from langgraph.types import interrupt

def human_node(state: State):
    value = interrupt(
        # Any JSON serializable value to surface to the human.
        # For example, a question or a piece of text or a set of keys in the state
       {
          "text_to_revise": state["some_text"]
       }
    )
    # Update the state with the human's input or route the graph based on the input.
    return {
        "some_text": value
    }

graph = graph_builder.compile(
    checkpointer=checkpointer # Required for `interrupt` to work
)

# Run the graph until the interrupt
thread_config = {"configurable": {"thread_id": "some_id"}}
graph.invoke(some_input, config=thread_config)

# Resume the graph with the human's input
graph.invoke(Command(resume=value_from_human), config=thread_config)
```
    </InterruptInfo>
    
    <Output>
    Unless explicitly mentioned in node requirements, human-in-loop aka interrupt is not needed in the scenario.
    If needed: implement the code with interrupt functionality tailored to the use case
    If not needed: just respond that interrupt functionality is not needed
    </Output>
    """

def interrupt_generation(state: NodeBuilderState) -> Command[Literal["ai_node_gen_supervisor"]]:
    """Generate the code for the node."""
    # fetch_docs=fetch_documents("https://langchain-ai.github.io/langgraph/concepts/human_in_the_loop/")
    response = llm.invoke( [SystemMessage(content=interrupt_gen_prompt)]  + state["messages"])
    return Command(
        update={
            "messages": [
                HumanMessage(content=response.content, name="interrupt_generator")
            ]
        },
        goto="ai_node_gen_supervisor",
    )


# Code compiler

In [114]:

code_compiler_prompt = ChatPromptTemplate.from_template("""
    You are tasked with furnishing a final output code for the user provided node information. 
    
    Analyze the message history, you will find multiple code pieces. which are related to prompt_gen, interrupt_gen, toolset_gen, structured_output_gen
    You need to carefully merge all the generate code together to furnish a final output.
    """)


def code_compiler(state: NodeBuilderState) -> Command[Literal["ai_node_gen_supervisor"]]:
    """Generate the code for the node."""
    response = llm.invoke(state["messages"])
    return Command(
        update={
            "messages": [
                HumanMessage(content=response.content, name="code_compiler")
            ]
        },
        goto="ai_node_gen_supervisor",
    )


In [115]:
ai_node_gen_supervisor = make_supervisor_node(llm, ["prompt_generation", "toolset_generation", "structured_output_generation", "interrupt_generation", "code_compiler"])

# Graph creation

In [116]:
from langgraph.graph.state import CompiledStateGraph
workflow = StateGraph(NodeBuilderState)
workflow.add_node("identify_node", identify_node)
workflow.add_node("deterministic_code", deterministic_code_gen)

workflow.add_node("ai_node_gen_supervisor", ai_node_gen_supervisor)
workflow.add_node("prompt_generation", prompt_generation)
workflow.add_node("toolset_generation", toolset_generation)
workflow.add_node("structured_output_generation", structured_output_generation)
workflow.add_node("interrupt_generation", interrupt_generation)
workflow.add_node("code_compiler", code_compiler)

workflow.add_edge(START, "identify_node")
workflow.add_conditional_edges("identify_node", determine_node_type, ["deterministic_code", "ai_node_gen_supervisor"])
workflow.add_edge("deterministic_code", END)

app : CompiledStateGraph = workflow.compile()

In [ ]:
from IPython.display import Image
display(Image(app.get_graph(xray=True).draw_mermaid_png()))

In [117]:
dict_for_node = {
    "planner": {
        "schema_info": """OverallState:
      type: TypedDict
      fields:
      - name: input
        type: str
      - name: plan
        type: List[str]
      - name: past_steps
        type: List[Tuple]
      - name: response
        type: str
      - name: messages
        type: Annotated[list[AnyMessage], add_messages]""",
    "input_schema": "OverallState",
    "output_schema":"OverallState",
    "description":"Plan step generates a plan based on the input using llm structured output functionality, stores it to the plan field",
    "function_name": "plan_step"
    },
    "math_node":{
        "schema_info": """MathState:
      type: TypedDict
      fields:
      - name: a
        type: int
      - name: b
        type: int
      - name: response
        type: int""",
    "input_schema": "MathState",
    "output_schema":"MathState",
    "description":"Gets the two numbers a and b from the state, and returns the sum of a and b., stores it to the response field",
    "function_name": "math_step"
    },
    "interrupt_node":{
        "schema_info": """RequirementAnalysisState:
      type: TypedDict
      fields:
      - name: user_input
        type: str
      - name: messages
        type: Annotated[list[AnyMessage], add_messages]""",
    "input_schema": "RequirementAnalysisState",
    "output_schema":"RequirementAnalysisState",
    "description":"Analyze the user input, see if they require any human intervention, if yes, return the human intervention required, else return no human intervention required",
    "function_name": "interrupt_step"
    },
    "decision_node":{
        "schema_info": """RequirementAnalysisState:
      type: TypedDict
      fields:
      - name: user_input
        type: str
      - name: messages
        type: Annotated[list[AnyMessage], add_messages]""",
    "input_schema": "RequirementAnalysisState",
    "output_schema":"RequirementAnalysisState",
    "description":"Analyze the user input which contain the requirements for building a product, acts as a decision node if more info is needed or not",
    "function_name": "decision_step"
    },
    "code_node":{
        "schema_info": """CodeWriterState:
      type: TypedDict
      fields:
      - name: user_query
        type: str
      - name: execution_result
        type: str""",
    "input_schema": "CodeWriterState",
    "output_schema":"RequiremenCodeWriterStatetAnalysisState",
    "description":"This node analyzes the user_query, if the query is to write a code, it will make a tool call to run the proposed code",
    "function_name": "code_step"
    },
    "weather_node":{
        "schema_info": """WeatherNodeState:
      type: TypedDict
      fields:
      - name: user_query
        type: str
      - name: execution_result
        type: str""",
    "input_schema": "WeatherNodeState",
    "output_schema":"WeatherNodeState",
    "description":"This node analyzes the user_query, makes tool calls to retrieve relevant information and answer the query",
    "function_name": "weather_step",
    },
    "stock_node":{
        "schema_info": """StockState:
      type: TypedDict
      fields:
      - name: user_query
        type: str
      - name: execution_result
        type: str""",
    "input_schema": "StockState",
    "output_schema":"StockState",
    "description":"This node analyzes the user_query related to finance/stock-markets, makes tool calls to retrieve relevant information and answer the query",
    "function_name": "stock_step",
    }
}

In [118]:
for output in app.stream(dict_for_node["stock_node"], stream_mode="updates"
    ):
        print(output)

{'identify_node': {'node_type': 'ai', 'messages': [HumanMessage(content='\nYou are provided with the following information about the node:\n<SchemaInfo>\nStockState:\n      type: TypedDict\n      fields:\n      - name: user_query\n        type: str\n      - name: execution_result\n        type: str\n</SchemaInfo>\n<InputSchema>\nStockState\n</InputSchema>\n<OutputSchema>\nStockState\n</OutputSchema>\n<Description>\nThis node analyzes the user_query related to finance/stock-markets, makes tool calls to retrieve relevant information and answer the query\n</Description>\n<FunctionName>\nstock_step\n</FunctionName>\n\nBelow is the skeleton of the function that you need to implement:\ndef stock_step(state:StockState) -> StockState:\n    """This node analyzes the user_query related to finance/stock-markets, makes tool calls to retrieve relevant information and answer the query"""\n    # Implement the function to meet the description.\n\nthe state is of type StockState and the function is of 